In [8]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import json
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'

In [44]:
cantidad = 100000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [45]:
path_to_json = 'output_seccionado/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))

In [46]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [47]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [48]:
get_closest(word= 'solidaridad', n = 3)

(['solidaridad', 'solidaria', 'insolidaridad', 'solidario'],
 [1, 0.5968679189682007, 0.5851333141326904, 0.5850723385810852])

In [49]:
model.similarity('educación', 'educar')

0.49962938

In [50]:
import es_core_news_md
nlp = es_core_news_md.load()
nlp_text = nlp('educado')
nlp_text[0].lemma_

'educar'

In [83]:
jsons[i]['experiencia'].count(word)

In [94]:
prc_description = '''ingeniería máster python excel desarrollo gestión comercial experiencia manejo clientes
emprendimiento liderar equipos planificar organizar '''

In [95]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        #print(similar_words[i], word_value[similar_words[i]])
        #print('------------------------------------------------')

In [102]:
word_value.keys()

dict_keys(['ingeniería', 'ingenierías', 'mecatrónica', 'máster', 'másteres', 'licenciatura', 'python', 'perl', 'scripts', 'excel', 'powerpoint', 'word', 'desarrollo', 'sostenible', 'socioeconómico', 'gestión', 'gestionar', 'planificación', 'comercial', 'comerciales', 'comercio', 'experiencia', 'experiencias', 'pericia', 'manejo', 'manejar', 'manejan', 'clientes', 'cliente', 'usuarios', 'emprendimiento', 'emprendimientos', 'emprendedor', 'liderar', 'encabezar', 'lidera', 'equipos', 'equipo', 'clubes', 'planificar', 'planear', 'planificarse', 'organizar', 'organizarlo'])

In [103]:
'Licencia' in jsons[1]['skills']

False

In [104]:
### ahora veamos si resulta el ranking
#frecuencia de término – frecuencia inversa de documento 
#Tf-idf
#Para calcular este ranking es mejor tener las secciones skills y experiencia
#con el fin de calcular esta metrica usando la ocurrencia de las palabras
# Tenemos todos los CV's y una descripción del cargo, a este descripcion del cargo
# tiene N palabras, le buscamos 2 palabras parecidas, generando una descripcion
# de N*2

# Usando esta nuevo set de palabras de descripcion, recorremos todos los cvs contando 
# la ocurrencia de estas palabras en cada documento, y luego se genera un ranking usando Tf-idf
# La pregunta es: ¿Lo haré sobre el documento entero? o ¿Trataré de seccionar y ocupar ciertas secciones?

no_of_cv = len(jsons)
print(no_of_cv)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        jsons[i]['skills'] = [x.lower() for x in jsons[i]['skills']]
        try:
            #if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
            if word in jsons[i]['skills'] or word in jsons[i]['experiencia'] or word in jsons[i]['educación']:
                #print('entre')
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
    print(idf)

34
{'ingeniería': 2.833213344056216}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616, 'licenciatura': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616, 'licenciatura': 3.5263605246161616, 'python': 3.5263605246161616}
{'ingeniería': 2.833213344056216, 'ingenierías': 3.52636

In [108]:

score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = jsons[i]['skills'].count(word) + jsons[i]['experiencia'].count(word) + jsons[i]['educación'].count(word)
            #print(tf)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass
print(score)

{0: 6.966126161669347, 1: 6.094712991655809, 2: 0.0, 3: 2.4369593107923397, 4: 6.182913570383523, 5: 3.4903986553945825, 6: 6.907389428145038, 7: 5.979010008567532, 8: 6.909261784044033, 9: 12.0946134443653, 10: 7.442058495821767, 11: 5.666426688112432, 12: 3.7899870980107915, 13: 0.9418802259307002, 14: 15.204158629608813, 15: 6.76922576108732, 16: 9.999526338307708, 17: 2.266307306603453, 18: 6.673344427705655, 19: 1.572574369157548, 20: 2.518570207056933, 21: 10.729645460559906, 22: 0.0, 23: 3.503883867357343, 24: 1.09073628226481, 25: 3.1898152210096535, 26: 8.31910658988543, 27: 7.326877068556623, 28: 16.511166372555003, 29: 20.368127020626453, 30: 5.969030654407657, 31: 3.3669619296708975, 32: 0.9636761737854768, 33: 7.554555625947028}


In [109]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['nombre archivo']))
    
sorted_list.sort(reverse = True)

#for s, i in sorted_list:
#    if list(cvs)[i] != '.DS_Store':
#        print(list(cvs)[i], ':', s)

In [110]:
sorted_list

[(20.368127020626453, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (16.511166372555003, '1581023751928-CV'),
 (15.204158629608813, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (12.0946134443653, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (10.729645460559906, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (9.999526338307708, '1580497335052-Jose_Medina_13-1-2020'),
 (8.31910658988543, '1583796411230-Curriculum__fco_marzo'),
 (7.554555625947028, '1586903416737-NewHTJ'),
 (7.442058495821767, '1582640494849-ANDRES_MENA_CV'),
 (7.326877068556623, '1582313013581-CV_FRF'),
 (6.966126161669347, '1582718980347-CV_-_Mario_Espinoza1'),
 (6.909261784044033,
  '1580231848251-Curriculum_Vitae_-_Jesús_Alonso_Vásquez_Cole'),
 (6.907389428145038, '1583871756395-CVCabanaGarayChristian1_1'),
 (6.76922576108732, '1581008153040-CV_Jose_Chávez'),
 (6.673344427705655, '1581690552715-CV_Gustavo_Martinez_Raya'),
 (6.182913570383523, '1581631090164-CV_ROSSANA_ALAYZA_-_FEBRERO_2020'),
 (6.094712991655809, '15809